# Promblem Statement:
As a data scientist, it's important to know how to scrape data from various sources, including websites. For this assignment, your task is to scrape the locations of your favourite retail brand in India and extract the following information:

* Store Name
* Address
* Timings
* Coordinates (Latitude/Longitude) (bonus)
* Phone Number
You can use any programming language of your choice, but we recommend using Python. Once you have extracted the data, you will need to store it in a CSV file and upload it to a GitHub repository alongside with the code.

Here are the steps to follow:

1. Identify your favourite retail brand with stores in India. Pick something that you enjoy buying things from.

2. Use web scraping techniques to extract the store locations, store name, address, timings, coordinates, and phone number from the website. It’s usually tricky to get the coordinates so it’s a fun challenge to see if you can do it.



# Importing required libraries

In [7]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

# Getting URL and responses using request.get method

In [10]:
# We choose spencers retail brand  to extract details 
url = 'http://www.spencersretail.com/store-list?page=2'

# Get request for URL to store the response
response = requests.get(url)

# Parse the HTML content of the response using BeautifulSoup and store the resulting object in a variable called Soup
soup = BeautifulSoup(response.text, 'html.parser')

# Extracting specific HTML Tag after that appending into list

In [11]:
# Find all the store names, addresses, localities, contact numbers, and opening/closing times using their respective HTML tags and attributes
store_name = soup.find_all("span", class_="fn")
store_address = soup.find_all("span", itemprop="streetAddress")
locality = soup.find_all("span", class_="locality")
Contact = soup.find_all("span", itemprop="telephone")
Time = soup.find_all("div", class_="field-content")

# Extract the opening times and closing times from the 'Time' variable and store them in separate lists called 'Opens_at' and 'Closes_at'
Opens_at=[]
for i in range(1,len(Time),3):
    Opens_at.append(Time[i])
    
Closes_at = []
for i in range(2,len(Time),3):
    Closes_at.append(Time[i])

# Create a list called 'full_address' containing dictionaries with the full address for each store, I will use this address list further to get the latitute and longitude
full_address=[]
for i in range(len(store_name)):
    full_address.append({
        store_address[i].text.strip() + postal_code[i].text.strip() + locality[i].text.strip() + region[i].text.strip()       
    })

In [12]:

# Now I will use an API from the wesite "Rapid API", which will give us the latitude and longitude of the addresses of the stores

# Set up the API endpoint URL and headers to access the TrueWay Geocoding API
url = "https://trueway-geocoding.p.rapidapi.com/Geocode"

headers = {
    "X-RapidAPI-Key": "b2dcd3ea06msh60d0300569d7b6dp1f52a6jsn31f7da53e46b",
    "X-RapidAPI-Host": "trueway-geocoding.p.rapidapi.com"
}


# For Loop for extract the latitude and longitude from the specified location according to our need.Here we create empty list called lat_long.
lat_long = []
for i in range(len(full_address)):
    querystring = {"address":full_address[i],"language":"en"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    lat_long.append((response.json()['results'][0]["location"]))
                  

# Create a list of dictionaries to store the extracted data for each store
data = []
for i in range(len(store_name)):
    data.append({
        "Store Name": store_name[i].text.strip(),
        "Store Address": store_address[i].text.strip(),
        "Locality": locality[i].text.strip(),
        "Contact": Contact[i].text.strip(),
        "Opens At": Opens_at[i].text.strip(),
        "Closes At": Closes_at[i].text.strip(),
        "Lat-Long": lat_long[i]
    })

# Storing all scraped data into structure format using Pandas Library

In [13]:
df = pd.DataFrame(data)

In [14]:
df

,Store Name,Store Address,Locality,Contact,Opens At,Closes At,Lat-Long
0,Spencer's Neighborhood Store,"Spencer Retail Limited,12/85,Club House Road",Kodaikanal,"241246, 242750",7:30 AM,21:30 PM,"{'lat': 10.238102, 'lng': 77.489168}"
1,Spencer's Hyper Store,"Spencer Retail Limited,Bellary Road - D. No. 5...",Kurnool,08518 - 256045,9:30 AM,22:00 PM,"{'lat': 15.139416, 'lng': 76.92142}"
2,Spencer's Hyper Store,"Spencer Retail Limited,D.No:- 69-4-21 , Pitapu...",Kakinada,0844 - 6456688,9:30 AM,22:00 PM,"{'lat': 17.075633, 'lng': 82.277821}"
3,Spencer's Hyper Store,"Spencer Retail Limited,Chinna Chowk D No. 42/3...",Kadapa,9247000206,9:30 AM,22:00 PM,"{'lat': 14.475417, 'lng': 78.820619}"
4,Spencer's Hyper Store,"Spencer Retail Limited,Basement, Pentagon Mall...",Haridwar,01334 232336,11:00 AM,21:30 PM,"{'lat': 29.947877, 'lng': 78.075473}"
5,Spencer's Neighborhood Store,"Spencer Retail Limited, Shop No : 2-4, Sector ...",Gurgaon,0124-4363254,10:00 AM,21:30 PM,"{'lat': 28.454575, 'lng': 77.039989}"
6,Spencer's Neighborhood Store,"Spencer Retail Limited,Sector - 4 & 7 Crossing...",Gurgaon,0124-6454725,10:00 AM,22:00 PM,"{'lat': 28.471078, 'lng': 77.011201}"
7,Spencer's Neighborhood Store,"Spencer Retail Limited,Sector - 07, Dharuhera ...",Gurgaon,,10:00 AM,21:30 PM,"{'lat': 28.205654, 'lng': 76.79148}"
8,Spencer's Neighborhood Store,"Spencer Retail Limited,SCF-78 Sec-14 main Market",Gurgaon,0124-6454284,10:00 AM,21:30 PM,"{'lat': 28.473523, 'lng': 77.047729}"
9,Spencer's Hyper Store,"Spencer Retail Limited,MGF Megacity Mall",Gurgaon,0124 - 4375331,10:00 AM,22:00 PM,"{'lat': 28.479734, 'lng': 77.08955}"


In [15]:
df.to_csv('Retail_Brand_data.csv', index=False)